In [ ]:
import os
import secrets
import urllib.request
import requests
from flask import Flask,render_template, request, jsonify, make_response, url_for
from flask_sqlalchemy import SQLAlchemy
from flask_bcrypt import Bcrypt
from werkzeug.utils import secure_filename
from itsdangerous import URLSafeTimedSerializer, SignatureExpired
from flask_cors import CORS
import jwt
from PIL import Image
import datetime
from functools import wraps

app = Flask(__name__)

app.config['SECRET_KEY'] = 'thisissecret'
app.config['SQLALCHEMY_DATABASE_URI'] = app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///todo.db'

UPLOAD_FOLDER = 'static/profile_pics'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
ALLOWED_EXTENSIONS = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])

bcrypt = Bcrypt(app)
db = SQLAlchemy(app)
s = URLSafeTimedSerializer('Thisisasecret!')
CORS(app)
cors = CORS(app, resources={r"/api/*": {"origins": "*"}})

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(50))
    email = db.Column(db.String(50))
    password = db.Column(db.String(80))
    admin = db.Column(db.Boolean)
    bio = db.Column(db.Text)
    image_file = db.Column(db.String(255), nullable=False, default='default.jpg')
    activated = db.Column(db.Boolean)
  

class Todo(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    text = db.Column(db.String(50))
    complete = db.Column(db.Boolean)
    user_id = db.Column(db.Integer)

def token_required(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        token = None
    
        if 'x-access-token' in request.headers:
            token = request.headers['x-access-token']
            print('token=',token)

        if not token:
            return jsonify({'message' : 'Token is missing!'}), 401

        try: 
            data = jwt.decode(token, app.config['SECRET_KEY'], algorithms=['HS256'])
            #data = jwt.decode(token, app.config['SECRET_KEY'])
            current_user = User.query.filter_by(id=data['id']).first()
            print("current_user=",current_user)
        except:
            return jsonify({'message' : 'Token is invalid!'}), 401

        return f(current_user, *args, **kwargs)

    return decorated


@app.route('/api/home')
def api_home():
    sqlalchemyObj = db.engine.execute('select * from venues WHERE front_page ORDER BY name')
    venues = []
    for i in sqlalchemyObj:
        venues.append(i)
    # print(venues)
    dataset = []
    dict={}
    desc_str = ''
    for i in venues:
        dict['Name'] = i[0]
        dict['Lat'] = i[1]
        dict['Lng'] = i[2]
        dict['Url'] = i[3]
        dict['Img_url'] = i[4]
        # desc_str = remove_html_tags(i[5])
        desc_str = i[5]
        dict['Desc'] = desc_str[:150]+"..."
        dict['front_page'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy())
    return jsonify({'Venues' : dataset})

@app.route('/api/destinations', methods=['GET']) # Get destination list
@token_required
def api_destinations(current_user):
    print('destinations-current_user:', current_user)
    sqlalchemyObj = db.engine.execute('select * from venues ORDER BY name')
    venues = []
    for i in sqlalchemyObj:
        venues.append(i)
    dataset = []
    dict={}
    for i in venues:
        dict['Name'] = i[0]
        dict['Lat'] = i[1]
        dict['Lng'] = i[2]
        dict['Url'] = i[3]
        dict['Img_url'] = i[4]
        dict['Desc'] = i[5]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) 
    # print(dataset)
    return jsonify({'Destinations' : dataset})


@app.route('/api/profile', methods=['GET',"PUT"]) 
@token_required
def api_profile(current_user):
    print('profile-current_user:', current_user)
    if request.method == 'PUT':
        data = request.get_json()
        picture_file = ''
        print('data:',data)
        user = User.query.filter_by(id=current_user.id).first()
        if data['image_file'] != '':
            picture_file = save_picture(data['image_file'])
            current_user.image_file = picture_file
            print("picture_file",picture_file)
        if user:
            print('data',data)
            user.username = data['username']
            hashed_password = bcrypt.generate_password_hash(data['password'])
            user.password = hashed_password
            user.email = data['email']
            user.bio =  data['bio']
            user.image_file =  picture_file
            db.session.commit()
            response = jsonify({'message' : 'The user has been updated!'}) 
            response.headers.add('Access-Control-Allow-Origin', '*')
            return response
            # return jsonify({'message' : 'The user has been updated!'}) 
        
    if request.method == 'GET':
       
        dataset = []
        user_data = {}
        user_data['id'] = current_user.id
        user_data['username'] = current_user.username
        user_data['email'] = current_user.email
        user_data['password'] = current_user.password.decode('utf-8')
        user_data['bio'] = current_user.bio
        print('current_user.id:', current_user.id)
        if current_user.image_file is None:
            current_user.image_file = 'default.jpg'
        image_file = url_for('static', filename='profile_pics/' + current_user.image_file)
        user_data['image_file'] = image_file
        dataset.append(user_data.copy())
        print('profile-dataset:',dataset)
        response = jsonify({'user' : dataset})
        response.headers.add('Access-Control-Allow-Origin', '*')
        return response


def save_picture(form_picture):
    print("form_picture",form_picture)
    random_hex = secrets.token_hex(8)
    _, f_ext = os.path.splitext(form_picture)
    picture_fn = random_hex + f_ext
    picture_path = os.path.join(app.root_path, 'static/profile_pics', picture_fn)
    output_size = (125, 125)
    temp_file = os.path.join(app.root_path, 'static/profile_pics', form_picture)
    i = Image.open(temp_file)
    i.thumbnail(output_size)
    i.save(picture_path)
    os.remove(temp_file)
    return picture_fn


def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/api/upload', methods=['POST'])
def upload_file():
    # check if the post request has the file part
    if 'file' not in request.files:
        resp = jsonify({'message' : 'No file part in the request'})
        resp.status_code = 400
        return resp
    file = request.files['file']
    if file.filename == '':
        resp = jsonify({'message' : 'No file selected for uploading'})
        resp.status_code = 400
        return resp
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        print("file name",filename)
        resp = jsonify({'message' : 'File successfully uploaded'})
        resp.status_code = 201
        return resp
    else:
        resp = jsonify({'message' : 'Allowed file types are txt, pdf, png, jpg, jpeg, gif'})
        resp.status_code = 400
        return resp

@app.route('/api/user', methods=['GET'])
@token_required
def get_all_users(current_user):
#def get_all_users():

    if not current_user.admin:
        return jsonify({'message' : 'Cannot perform that function!'})

    users = User.query.all()

    output = []

    for user in users:
        user_data = {}
        print(type(user.username))
        print(type(user.password))
        user_data['username'] = user.username
        user_data['email'] = user.email
        user_data['password'] = str(user.password)
        user_data['admin'] = user.admin
        output.append(user_data)

    return jsonify({'users' : output})

@app.route('/api/user/<id>', methods=['GET'])
@token_required
def get_one_user(current_user, id):
    print("current_user:",current_user,"id:",id)
    if not current_user.admin:
        return jsonify({'message' : 'Cannot perform that function!'})

    user = User.query.filter_by(id=id).first()

    if not user:
        return jsonify({'message' : 'No user found!'})

    user_data = {}
    user_data['id'] = user.id
    user_data['username'] = user.username
    user_data['email'] = user.email
    user_data['password'] = user.password
    user_data['admin'] = user.admin

    return jsonify({'user' : user_data})

@app.route('/api/register', methods=['POST'])
def register():
    data = request.get_json()
    hashed_password = bcrypt.generate_password_hash(data['password'])
    new_user = User(username=data['username'], email=data['email'], password=hashed_password, admin=False)
    token = s.dumps(data['email'], salt='email-confirm')
    link = url_for('confirmemail', token=token, _external=True)       
    message = "Click the link to confirm your registration:"+link
    username = data['username']
    email = data['email']
    registered = True

    mydict =  {
        "name": username,
        "email": email,
        "registered": registered,
        "message": message
        }

    data = {"fields": mydict}
    headers = {'Authorization': 'Bearer xxxxxxxxxxxx', 'Content-Type': 'application/json; charset=utf-8'}
    # r = requests.post(url = API_ENDPOINT, data = data)
    r = requests.post('https://api.airtable.com/v0/appajvHoXUME1ZmI2/sendmail',json=data,headers=headers)
    print("Status Code:",r.status_code)
    print(r.text)
    
    db.session.add(new_user)
    db.session.commit()
    return jsonify({'message' : 'Registration completed!'})

@app.route('/confirmemail/<token>')
def confirmemail(token):
    try:
        email  = s.loads(token, salt='email-confirm', max_age=3600)
        user = User.query.filter_by(email=email).first()
        if user:
            user.activated = True
            db.session.commit()
        # return 'The token works!'
        message = 'Thank you for authenticating your email!'
    except SignatureExpired:
        # return '<h1>The confirmation token is expired!</h1>'
        message = 'The confirmation token is expired!'

    return render_template('confirmation.html', message=message)


@app.route('/api/user/<id>', methods=['POST'])
@token_required
def create_user(current_user, id):
    if not current_user.admin:
        return jsonify({'message' : 'Cannot perform that function!'}) 
    data = request.get_json()
    hashed_password = bcrypt.generate_password_hash(data['password'])
    new_user = User(username=data['username'], email=data['email'], password=hashed_password, admin=False)
    db.session.add(new_user)
    db.session.commit()
    return jsonify({'message' : 'New user created!'})

@app.route('/api/user/<id>', methods=['PUT'])
@token_required
def promote_user(current_user, id):
    print(current_user)
    if not current_user.admin:
        return jsonify({'message' : 'Cannot perform that function!'})

    user = User.query.filter_by(id=id).first()

    if not user:
        return jsonify({'message' : 'No user found!'})

    user.admin = True
    db.session.commit()

    return jsonify({'message' : 'The user has been promoted!'})

@app.route('/api/user/<id>', methods=['DELETE'])
@token_required
def delete_user(current_user, id):
    if not current_user.admin:
           return jsonify({'message' : 'Cannot perform that function!'})

    user = User.query.filter_by(id=id).first()

    if not user:
        return jsonify({'message' : 'No user found!'})

    db.session.delete(user)
    db.session.commit()

    return jsonify({'message' : 'The user has been deleted!'})

@app.route('/api/login')
def login():
    auth = request.authorization

    if not auth or not auth.username or not auth.password:
        return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

    user = User.query.filter_by(username=auth.username).first()

    if not user:
        return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

    if bcrypt.check_password_hash(user.password, auth.password) and user.activated:
        token = jwt.encode({'id' : user.id, 'exp' : datetime.datetime.utcnow() + datetime.timedelta(minutes=30)}, app.config['SECRET_KEY'])

        # return jsonify({'token' : token.decode('UTF-8')}) for PythonAnywhere version
        return jsonify({'token' : token})

    return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

@app.route('/api/todo', methods=['GET'])
@token_required
def get_all_todos(current_user):
    todos = Todo.query.filter_by(user_id=current_user.id).all()

    output = []

    for todo in todos:
        todo_data = {}
        todo_data['id'] = todo.id
        todo_data['text'] = todo.text
        todo_data['complete'] = todo.complete
        output.append(todo_data)

    return jsonify({'todos' : output})

@app.route('/api/todo/<todo_id>', methods=['GET'])
@token_required
def get_one_todo(current_user, todo_id):
    todo = Todo.query.filter_by(id=todo_id, user_id=current_user.id).first()

    if not todo:
        return jsonify({'message' : 'No todo found!'})

    todo_data = {}
    todo_data['id'] = todo.id
    todo_data['text'] = todo.text
    todo_data['complete'] = todo.complete

    return jsonify(todo_data)

@app.route('/api/todo', methods=['POST'])
@token_required
def create_todo(current_user):
    data = request.get_json()

    new_todo = Todo(text=data['text'], complete=False, user_id=current_user.id)
    db.session.add(new_todo)
    db.session.commit()

    return jsonify({'message' : "Todo created!"})

@app.route('/api/todo/<todo_id>', methods=['PUT'])
@token_required
def complete_todo(current_user, todo_id):
    todo = Todo.query.filter_by(id=todo_id, user_id=current_user.id).first()
    
    if not todo:
        return jsonify({'message' : 'No todo found!'})
    data = request.get_json()
    todo.complete = True
    todo.text = data['text']
    db.session.commit()

    return jsonify({'message' : 'Todo item has been completed!'})

@app.route('/api/todo/<todo_id>', methods=['DELETE'])
@token_required
def delete_todo(current_user, todo_id):
    todo = Todo.query.filter_by(id=todo_id, user_id=current_user.id).first()

    if not todo:
        return jsonify({'message' : 'No todo found!'})

    db.session.delete(todo)
    db.session.commit()

    return jsonify({'message' : 'Todo item deleted!'})

if __name__ == '__main__':
    app.run('localhost', port=9018)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/suen/Documents/anaconda3/lib/python3.9/site-packages/flask_sqlalchemy/__init__.py:868: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9018/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Jan/2023 15:00:29] "GET /api/home HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2023 15:00:45] "OPTIONS /api/register HTTP/1.1" 200 -


Status Code: 200
{"id":"rec3bvxUW6XyycfFT","createdTime":"2023-01-30T07:00:47.000Z","fields":{"name":"bsuen","email":"bernard@cuhk.edu.hk","registered":true,"message":"Click the link to confirm your registration:http://localhost:9018/confirmemail/ImJlcm5hcmRAY3Voay5lZHUuaGsi.Y9drHQ.2WcbhCNDc1SIgvv2IUDVMUmw36Y","record_id":"rec3bvxUW6XyycfFT"}}


[2023-01-30 15:00:52,035] ERROR in app: Exception on /api/register [POST]
Traceback (most recent call last):
  File "/Users/suen/Documents/anaconda3/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 1819, in _execute_context
    self.dialect.do_execute(
  File "/Users/suen/Documents/anaconda3/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 732, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: database is locked

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/suen/Documents/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/suen/Documents/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/suen/Documents/anaconda3/lib/python3.9/site-packages/flask_cors/extension.py", line 165, in wrap

Status Code: 200
{"id":"recEMYBW7vPPJWfax","createdTime":"2023-01-30T07:02:31.000Z","fields":{"name":"bsuen","email":"bernard@cuhk.edu.hk","registered":true,"message":"Click the link to confirm your registration:http://localhost:9018/confirmemail/ImJlcm5hcmRAY3Voay5lZHUuaGsi.Y9drhg.oLZxliCsoQnh2dJO9MUf2Kskkmk","record_id":"recEMYBW7vPPJWfax"}}


...

127.0.0.1 - - [30/Jan/2023 15:18:17] "OPTIONS /api/register HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2023 15:18:18] "POST /api/register HTTP/1.1" 200 -


Status Code: 200
{"id":"rec1LtfWMokDnOKnU","createdTime":"2023-01-30T07:18:19.000Z","fields":{"name":"david","email":"david@hkdesign.org","registered":true,"message":"Click the link to confirm your registration:http://localhost:9018/confirmemail/ImRhdmlkQGhrZGVzaWduLm9yZyI.Y9dvOQ.QLhU1YR6LiWYtMor4__yaS77CyY","record_id":"rec1LtfWMokDnOKnU"}}


127.0.0.1 - - [30/Jan/2023 15:19:49] "OPTIONS /api/register HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2023 15:19:51] "POST /api/register HTTP/1.1" 200 -


Status Code: 200
{"id":"recEHHNdzyoW7Kd8L","createdTime":"2023-01-30T07:19:51.000Z","fields":{"name":"support","email":"support@intechnigence.com","registered":true,"message":"Click the link to confirm your registration:http://localhost:9018/confirmemail/InN1cHBvcnRAaW50ZWNobmlnZW5jZS5jb20i.Y9dvlg.5Wj7yhTRZaMu87R01BcQBn_dqTM","record_id":"recEHHNdzyoW7Kd8L"}}


127.0.0.1 - - [30/Jan/2023 15:20:34] "GET /confirmemail/ImRhdmlkQGhrZGVzaWduLm9yZyI.Y9dvOQ.QLhU1YR6LiWYtMor4__yaS77CyY HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2023 15:20:34] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Jan/2023 15:21:26] "GET /confirmemail/InN1cHBvcnRAaW50ZWNobmlnZW5jZS5jb20i.Y9dvlg.5Wj7yhTRZaMu87R01BcQBn_dqTM HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2023 15:24:37] "OPTIONS /api/login HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2023 15:24:37] "GET /api/login HTTP/1.1" 200 -
